In [ ]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from abelfunctions import RiemannTheta

import layers as layers
import model as mdl

import warnings
warnings.filterwarnings('ignore')

from costfunctions import mse
import minimizer

# Example 1: Sine-Mixture with gaussian noise

In [ ]:
def func(x,n):
    return 0.5*np.sin(1*x+0.1)+0.75*np.sin(0.25*x-0.3)+n*np.random.normal(0,0.2,x.shape[2])

X = np.linspace(0, 100, 500)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))

In [ ]:
plt.plot(X.flatten(), Y.flatten(),"ob-")

In [ ]:
M = mdl.Model()
M.add(layers.ThetaUnitLayer(1,3))
M.add(layers.ThetaUnitLayer(3,3))
M.add(layers.ThetaUnitLayer(3,1))

In [ ]:
minim = minimizer.CMA(parallel=True)
minim.train(mse, M, X, Y, tolfun=1e-5)

In [ ]:
npoints = 500
test_X = (np.linspace(0, 100, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"ob-", label='fit')
plt.legend()

# Example 2: Sine-Cosine-Mixture with trend and gaussian noise

In [ ]:
def func(x,n):
    return 0.02*x+0.5*np.sin(1*x+0.1)+0.75*np.cos(0.25*x-0.3)+n*np.random.normal(0,0.2,x.shape[2])

X = np.linspace(0, 100, 500)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))

In [ ]:
plt.plot(X.flatten(), Y.flatten(),"ob-")

In [ ]:
M = mdl.Model()

M.add(layers.ThetaUnitLayer(1,3, param_bound=10))
M.add(layers.ThetaUnitLayer(3,3, param_bound=10))
M.add(layers.ThetaUnitLayer(3,2, param_bound=10))
M.add(layers.ThetaUnitLayer(2,1, param_bound=10))

In [ ]:
minim = minimizer.CMA(parallel=True)
minim.train(mse, M, X, Y, tolfun=1e-5)

In [ ]:
npoints = 500
test_X = (np.linspace(0, 100, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"ob-", label='fit')
plt.legend()

# Example 3: MNIST

In [2]:
# Load MNIST dataset
MNIST_train = pd.read_csv('~/data/mnist_train.csv', delimiter=",",header=None).values
MNIST_test  = pd.read_csv('~/data/mnist_test.csv', delimiter=",",header=None).values

# Prepare data (normalized onto [0,1])
Y_train = MNIST_train[0:100,0]
X_train = MNIST_train[0:100,1:]/255.0

Y_test = MNIST_test[:,0]
X_test = MNIST_test[:,1:]/255.0

In [3]:
M = mdl.Model()
M.add(layers.ThetaUnitLayer(784,10))
M.add(layers.ThetaUnitLayer(10,1))



In [ ]:
minim = minimizer.CMA(True)

minim.train(mse,M, np.transpose(X_train), np.transpose(Y_train),tolfun=1e-1)

CMA on 16 cpu(s) enabled
(15_w,30)-aCMA-ES (mu_w=8.6,w_1=20%) in dimension 7872 (seed=266724, Mon Oct 16 16:52:29 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     30 1.379030483804791e+01 1.0e+00 9.99e-01  1e+00  1e+00 0:02.6
    2     60 1.228871841881150e+01 1.0e+00 9.97e-01  1e+00  1e+00 0:05.0
    3     90 1.367826027754914e+01 1.0e+00 9.96e-01  1e+00  1e+00 0:07.5
    5    150 9.551029459186095e+00 1.0e+00 9.94e-01  1e+00  1e+00 0:12.4
    7    210 7.538809439544775e+00 1.0e+00 9.92e-01  1e+00  1e+00 0:17.3
   10    300 5.474190035568929e+00 1.0e+00 9.88e-01  1e+00  1e+00 0:24.7
   13    390 6.865447146524842e+00 1.0e+00 9.85e-01  1e+00  1e+00 0:32.2
   16    480 6.119037069966669e+00 1.0e+00 9.82e-01  1e+00  1e+00 0:39.7
   20    600 5.221360972225561e+00 1.0e+00 9.78e-01  1e+00  1e+00 0:49.6
   24    720 5.432798089907346e+00 1.0e+00 9.74e-01  1e+00  1e+00 0:59.5
   29    870 4.497418201755291e+00 1.0e+00 9.69e-01  1e+00  1e+00 1:11.7
   34

In [ ]:
P=np.real(M.predict(np.transpose(X_test)))

where_are_NaNs = np.isnan(P)
P[where_are_NaNs] = 0
P = np.abs(np.round(P))
P=P.reshape(10000,)

print(classification_report(Y_test,P))
print(confusion_matrix(Y_test, P))